In [1]:
import pandas as pd
import numpy as np
import json
import time
import requests
from uszipcode import SearchEngine

In [2]:
# Thanks to Justin August at https://github.com/justinaugust/what-value-what-damage
def get_quandl(zipcode, index):    
    
    base_url = 'https://www.quandl.com/api/v3/datasets/ZILLOW/'
    
    #you need to enter your own API KEY
    api_key = "enter your api key here"
    
    q_code = 'Z' + zipcode + '_' + index
    ftype = '.json'
    params = {
            'api_key' : api_key,
        }
    get_url = base_url+q_code+ftype
    res = requests.get(get_url, params)
        
    return res.json()

In [3]:
zipcodes = list(pd.read_csv('../data/zip.csv', dtype={'ID': object}, header=0))

In [4]:
# The Zillow Home Value Index is Zillow's estimate of the median market value of zillow home value index
# - all homes within the zip of 02114, Boston, MA. This data is calculated by Zillow Real Estate Research
# (www.zillow.com/research) using their database of 110 million homes.
# ZHVIAH: Zillow Home Value Index - All Homes
# ZHVIBT: Zillow Home Value Index - Bottom Tier
# ZHVIMT: Zillow Home Value Index - Middle Tier
# ZHVITT: Zillow Home Value Index - Top Tier
# MVALFAH: Median Value Per Square Foot - All Homes

indexes = ['ZHVIAH', 'ZHVIBT', 'ZHVIMT', 'ZHVITT', "MVALFAH"]

data = {}
val_data = []

for zipcode in zipcodes:
    data[zipcode] = {}
    row = []
    row.append(zipcode)
    
    for idx in indexes:
        quandl_data = get_quandl(zipcode, idx)
        if 'quandl_error' in quandl_data.keys():
            data[zipcode][idx] = {}
            row.append(np.nan)            
        else: 
            data[zipcode][idx] = quandl_data['dataset']
            row.append(data[zipcode][idx]['data'][0][1])
        
    val_data.append(row)

In [5]:
# val_data
col = ['zip'] + indexes
df_value = pd.DataFrame(val_data, columns=col)
df_value

,zip,ZHVIAH,ZHVIBT,ZHVIMT,ZHVITT,MVALFAH
0,02110,1334048.0,828293.0,1334048.0,2628599.0,1138.0
1,02196,NaN,NaN,NaN,NaN,NaN
2,02201,NaN,NaN,NaN,NaN,NaN
3,02204,150686.0,NaN,150686.0,NaN,82.0
4,02206,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
181,02052,686738.0,539163.0,686738.0,927751.0,307.0
182,01965,NaN,NaN,NaN,NaN,NaN
183,02067,574371.0,451061.0,574371.0,771123.0,674.0
184,01770,804986.0,644396.0,804986.0,1083530.0,662.0


In [6]:
df_value.dropna()

,zip,ZHVIAH,ZHVIBT,ZHVIMT,ZHVITT,MVALFAH
0,02110,1334048.0,828293.0,1334048.0,2628599.0,1138.0
7,02241,93464.0,65011.0,93464.0,190798.0,59.0
18,02109,901207.0,628513.0,901207.0,1895015.0,858.0
21,02113,608509.0,462356.0,608509.0,914301.0,866.0
22,02108,1310395.0,643204.0,1310395.0,3779910.0,1138.0
...,...,...,...,...,...,...
180,02081,518835.0,414728.0,518835.0,707868.0,298.0
181,02052,686738.0,539163.0,686738.0,927751.0,307.0
183,02067,574371.0,451061.0,574371.0,771123.0,674.0
184,01770,804986.0,644396.0,804986.0,1083530.0,662.0


In [7]:
df_value.to_csv('../data/zillow_w_na.csv', index=False)

In [8]:
df_value.dropna().to_csv('../data/zillow.csv', index=False)